## Mall Customer Segmentation

Use the given customer credentials to determine which customers are suitable becoming your target customer.

The data is from membership cards which provide Customer ID, age, gender, annual income, and spending score.

Spending Score is something you assign to the customer based on your defined parameters like customer behavior and purchasing data.

### 1. Import Libraries

### 2. Data Loading

### 3. Data Cleaning

### 4. Exploratory Data Analysis

### 5. Feature Engineering
- do feature transformation (scaling, PCA)
- compare PCA features vs original features.

### 6. Modeling
**Clustering algorithm options**:  

  a. K-Means: simple and easy to interpret with assumption of spherical clusters. It works best with numeric dataset, but gender can be transformed with one-hot encoding. 
    
  b. DBScan: suitable when you expect clusters vary in sizes and shapes.  
   
  c. Gaussian Mixture Model: appropriate when features have different distributions. It's a probabilistic modeling approach.

### 7. Model Evaluation

**Evaluation metrics options**:  

 a. Silhouette Score: measures similarity of a point to its cluster compared to other clusters.  
 - The score range is from -1 to +1. Score near to +1 indicates points are well-clustered.  

 b. Davies-Bouldin Index: measures ratio of average intra-cluster distance to average inter-cluster distance. The smaller, the better the clustering since the separation is more apparent.  
  - <u>intra-cluster distance</u> shows how compact your cluster is (high internal similarity).  
  - <u>inter cluster distance</u> describes how different a cluster compared to another one (high external differentiation).

### 7. Conclusion